In [1]:
import json
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text

In [2]:
selected_brands = pd.read_csv("selected_brands.csv")

# read and save yelp data

In [3]:
data = []
with open("../SafeGraph_data/yelp_dataset/yelp_academic_dataset_business.json") as f:
    for line in f:
        aline = json.loads(line)
        
        try:
            price = aline['attributes']['RestaurantsPriceRange2']
            name = aline['name']
            data.append([name, price])
        except:
            next

brand_yelp = pd.DataFrame(data,  columns = ['brands', 'yelp_price'])
brand_yelp = brand_yelp[brand_yelp['yelp_price'] != 'None']

In [4]:
len(brand_yelp)

85280

In [5]:
mode = brand_yelp.groupby('brands').apply(lambda x: stats.mode(x['yelp_price'], keepdims = False)[0][0])
brand_yelp_raw = pd.DataFrame({"yelp_dollar": mode})
brand_yelp_raw = brand_yelp_raw.reset_index()

/var/folders/3n/5f66kdbs79l01jvps_16tnh00000gn/T/ipykernel_29884/1825127623.py:1: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  mode = brand_yelp.groupby('brands').apply(lambda x: stats.mode(x['yelp_price'], keepdims = False)[0][0])


In [6]:
brand_yelp_raw

,brands,yelp_dollar
0,Joe's Throwback Barber Shop,1
1,Leland's Barbershop,2
2,"""Genuino"" Italian Cafe'",2
3,# 1 Nail Salon,2
4,#1 Mongolian BBQ - Best Stir Fried Noodles In ...,2
...,...,...
60419,¡Juice!,2
60420,ÀLAVITA,2
60421,Àrdana Food & Drink,2
60422,Éclat Chocolate,3


In [63]:
brand_yelp_raw.to_excel('temp_data/brand_yelp_row.xlsx', index = False)

# match and manual labell the unlabelled

In [3]:
brand_yelp_raw = pd.read_excel('temp_data/brand_yelp_row.xlsx')

In [4]:
matched = pd.merge(selected_brands, brand_yelp_raw, on = 'brands')

In [5]:
matched

,brands,yelp_dollar
0,1-800-Flowers,2
1,7-Eleven,1
2,ALDI,1
3,Acme Markets,2
4,Allen Edmonds,3
...,...,...
363,Zara,2
364,Zumiez,2
365,honeygrow,2
366,lululemon athletica,3


In [6]:
unlabelled = set(selected_brands['brands']) - set(matched['brands'])
unlabelled = pd.DataFrame({'brands': list(unlabelled)})
unlabelled = unlabelled.sort_values('brands').reset_index(drop = True)
unlabelled

,brands
0,& Other Stories
1,&pizza
2,10 Spot/MadRag
3,110 Grill
4,16 Handles
...,...
551,itsu
552,junzi
553,maurices
554,rue21


In [25]:
# # save data with csv
# unlabelled.to_csv('temp_data/unlabelled.csv', index = False)
# # do the manual label with xlsx
# unlabelled.to_excel('temp_data/to_manual.xlsx', index = False) 

# read in and add manual labelled data

In [7]:
to_manual = pd.read_excel("temp_data/to_manual.xlsx")

In [8]:
to_manual

,brands,yelp_dollar
0,& Other Stories,2.0
1,&pizza,2.0
2,10 Spot/MadRag,2.0
3,110 Grill,2.0
4,16 Handles,1.0
...,...,...
551,itsu,2.0
552,junzi,2.0
553,maurices,2.0
554,rue21,2.0


In [9]:
yelp_labelled = pd.concat([matched, to_manual])
yelp_labelled = yelp_labelled.sort_values('brands')

In [10]:
yelp_labelled

,brands,yelp_dollar
0,& Other Stories,2.0
1,&pizza,2.0
0,1-800-Flowers,2.0
2,10 Spot/MadRag,2.0
3,110 Grill,2.0
...,...,...
366,lululemon athletica,3.0
553,maurices,2.0
554,rue21,2.0
367,sweetgreen,2.0


In [11]:
# yelp_labelled.to_csv("yelp_labelled.csv", index = False)

In [12]:
sum(yelp_labelled['yelp_dollar'].isna())

148

In [13]:
sum(yelp_labelled['yelp_dollar'].notna())

776